# ATTAINS DATA EXPLORATION
<i> Ryan Treves

Under the Clean Water Act (CWA) section 303(d), states and Tribes are required to monitor water bodies in their territory, and identify water bodies for which a water quality standard has not been met. To do this, they must conduct 'assessments' of water bodies based on whether that water body attains its water quality standard for a certain type of use (e.g., swimming, fishing). Water bodies are delineated into 'assessment units' for the purposes of organizing assessments. Based on the results of the assessment, an assessment unit is assigned an Integrated Reporting (IR) category from 1-5 based on the degree of impairment of the water body. An IR category 5 determination means that a TMDL should be developed for that assessment unit, and the water is 'impaired'. (See https://drive.google.com/drive/u/0/folders/1tGpCSD-3mRBChTC1PZStt5bdRljTK1ck for more details on IR category determination).
 Every two years (reporting cycles are biennial), states must submit their list of impaired waters (including but not limited to all waters with an IR category 5 determination) as a part of their Integrated Report (IR) to the federal EPA under  sections 303(d), 305(b), and 314.

See https://www.epa.gov/wqs-tech/supplemental-module-listing-impaired-waters-and-developing-tmdls#tab-2  for more details.

### Questions:
- How many Assessment Units (AUs) exist nationwide?
- For many AUs do we have a HUC code match?
- How many use assessments nationwide, ever, have contributed to an IR5 category determination?
- How many unique assessment units have been assigned category IR5?
- Which states have had the most use assessments leading to IR5 determinations?
- Which states have had the highest rate of IR5 determinations per assessment unit?
- For what fraction of use assessments do we have an assessment date?
- What parameters have caused the most use non-attainment declarations?
- What parameters have caused the most assessment units to be categorized as IR5, irrespective of number of use non-attainment declarations?
- What uses have the highest rate of non-attainment?

Note: the dataset of AUs doesn't include Pennsylvania (see `ATTAINS_data_cleaning.ipynb` for an explanation)


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import json
from urllib.request import urlopen
import datetime as dt
import warnings

# Suppress warning messages
warnings.filterwarnings('ignore')
pd.options.display.max_columns = 200
pd.options.display.max_rows = 2000
# display all rows & columns
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# Assessment Units

In [ ]:
# Load in national Assessment Unit (AU) data
AUs = pd.read_csv('Clean_AU_data/all_AUs_cleaned.csv')

In [ ]:
AUs.head()

### How many Assessment Units (AUs) exist nationwide?
Note: this estimate doesn't include Pennsylvania, which according to https://attains.epa.gov/attains-public/api/assessmentUnits?stateCode=PA&returnCountOnly=Y contains on the order of 200,000 AUs on its own.

In [ ]:
len(AUs['AUID'].unique())

### For many AUs do we have a HUC code match?

In [ ]:
AUs[(~pd.isna(AUs['HUC-12'])) | (~pd.isna(AUs['HUC-10'])) | (~pd.isna(AUs['HUC-8']))].shape[0]

Note - that fact that we don't have HUC code matches for every AU may not be as big an issue as it seems.

### Does a single water body ever have more than one AUID on record?

In [ ]:
counts = AUs.groupby(['items.assessmentUnits.assessmentUnitName', 'items.assessmentUnits.assessmentUnitIdentifier'], as_index=False).nunique()
changed_AUIDs = counts[counts['AUID']>1][['items.assessmentUnits.assessmentUnitName', 'items.assessmentUnits.assessmentUnitIdentifier']]

In [222]:
changed_AUIDs = changed_AUIDs.merge(AUs[['AUID', 'HUC-12', 'items.assessmentUnits.assessmentUnitName', 'items.assessmentUnits.assessmentUnitIdentifier']], on=['items.assessmentUnits.assessmentUnitName', 'items.assessmentUnits.assessmentUnitIdentifier'], how='left').drop_duplicates()
changed_AUIDs

,items.assessmentUnits.assessmentUnitName,items.assessmentUnits.assessmentUnitIdentifier,AUID_x,HUC-12_x,AUID_y,HUC-12_y
0,Aarons Creek from the confluence with Big Bran...,Aarons Creek,VAC-L73R_AAR01A00,NaN,VAC-L73R_AAR01A00,NaN
2,Aarons Creek from the confluence with Big Bran...,Aarons Creek,VAC-L73R_AAR01A00,NaN,VAW-L73R_AAR01A00,NaN
3,Aarons Creek from the confluence with Big Bran...,Aarons Creek,VAW-L73R_AAR01A00,NaN,VAC-L73R_AAR01A00,NaN
5,Aarons Creek from the confluence with Big Bran...,Aarons Creek,VAW-L73R_AAR01A00,NaN,VAW-L73R_AAR01A00,NaN
6,Abenaki,ABENAKI,VT14-03L03_02,NaN,VT14-03L03_02,NaN
7,Abenaki,ABENAKI,VT14-03L03_02,NaN,VT14-03L03_01,NaN
8,Abenaki,ABENAKI,VT14-03L03_01,NaN,VT14-03L03_02,NaN
9,Abenaki,ABENAKI,VT14-03L03_01,NaN,VT14-03L03_01,NaN
10,All waters in watershed L33R,Unsegmented Portion of Watershed L33,VAC-L33R_ZZZ01A00,NaN,VAC-L33R_ZZZ01A00,NaN
12,All waters in watershed L33R,Unsegmented Portion of Watershed L33,VAC-L33R_ZZZ01A00,NaN,VAW-L33R_ZZZ01A00,NaN


Cross-referencing these records with the assessments file below, it appears that many of the examples in the table above represent water bodies in Virginia that underwent a AUID change in 2018 from using the 'VAC' prefix to using 'VAP' or 'VAW' (maybe this has to do with a change to the state's segmentation methodology?). Similar renamings may have occurred in CT, RI, MD, and MO.
Other examples seem to be distinct water bodies that happen to share the same `items.assessmentUnits.assessmentUnitName` and `items.assessmentUnits.assessmentUnitIdentifier` values - in some cases, because they are unnamed tributaries or unsegmented portions of watersheds, in other cases, because names are not specific. This appears to be the case for the examples from Alabama.
Some examples from Vermont use a two-digit 'Assessment Database segment code' suffix for "EPA tracking purposes" (e.g., VT17-01L01_01 and VT17-01L01_02, see https://www.epa.gov/sites/default/files/2019-02/documents/2018-vt-303d-list-report.pdf)

# Assessments
Source: `pull_NotSupporting_assessments.R`

In [ ]:
assessments = pd.read_csv('all_NotSupporting_assessments.csv', dtype={'reportingCycleText': str}, parse_dates=['assessment_date'], date_parser=lambda t: pd.to_datetime(t, errors='coerce'))
assessments.head()

In [ ]:
len(assessments)

### How many use assessments nationwide, ever, have contributed to a 'Not Supporting' determination?
From here on, 'impaired' = received a 'Not Supporting' determination for any use. One assessment unit can be impaired by having multiple impairments - e.g., multiple chemicals (parameters) or multiple uses or both.
Here, a use assessment is uniquely identified by assessmentUnitIdentifier + useName + parameterName + reportingCycleText + assessment_date

In [ ]:
assessments.drop_duplicates(subset=['assessmentUnitIdentifier', 'useName', 'reportingCycleText', 'parameterName', 'assessment_date']).shape[0]

These numbers are different because the assessments table contains one row per use assessment x TMDL instance, and some use assessments are associated with multiple TMDLs from different years.

### What proportion of assessments have been associated with multiple TMDLs?

In [ ]:
assessments.duplicated(subset=['assessmentUnitIdentifier', 'useName', 'reportingCycleText', 'parameterName', 'assessment_date']).value_counts(normalize=True)

### What proportion of all assessment units have been associated with multiple TMDLs?

In [ ]:
len(assessments[assessments.duplicated(subset=['assessmentUnitIdentifier', 'associatedActionIdentifier'])].dropna(subset='associatedActionIdentifier').drop_duplicates('assessmentUnitIdentifier'))/len(AUs['AUID'].unique())

### How many unique assessment units have been impaired?

In [ ]:
len(assessments['assessmentUnitIdentifier'].unique())

### Which states have had the most use assessments leading to impairments?

In [ ]:
assessments.drop_duplicates(subset=['assessmentUnitIdentifier', 'useName', 'reportingCycleText',  'parameterName', 'assessment_date'])[['state_code']].value_counts(normalize=True)

### Which states have the highest and lowest rates of impairments per assessment unit?

In [ ]:
# Get counts of assessment units in each state
AU_counts = {}
for state in assessments['state_code'].unique():
    response = urlopen('https://attains.epa.gov/attains-public/api/assessmentUnits?stateCode=' + state + '&returnCountOnly=Y')
    data = json.loads(response.read())['count']
    AU_counts[state] = data

In [ ]:
rates = pd.DataFrame(assessments.drop_duplicates(subset=['assessmentUnitIdentifier', 'useName',  'parameterName', 'reportingCycleText', 'assessment_date'])['state_code'].value_counts())
rates = rates.reset_index().rename(columns = {'index':'state', 'state_code':'# impairments'})
rates['AUs'] = rates['state'].apply(lambda x: AU_counts[x])
rates['Impairment rate'] = rates['# impairments']/rates['AUs']

In [ ]:
rates.sort_values(by='Impairment rate', ascending=False).head(10)

In [ ]:
rates.sort_values(by='Impairment rate', ascending=False).tail(10)

Note: this table counts an impairment as a unique combination of a parameter x use x AUID x reportingCycle. Thus, having one parameter lead to 5 uses being impaired will have the same effect on impairment rate as 5 parameters each leading to one use being impaired.

### For what fraction of use assessments do we have an assessment date?

In [ ]:
use_assessments_unique = assessments.drop_duplicates(subset=['assessmentUnitIdentifier', 'useName', 'parameterName', 'reportingCycleText', 'assessment_date'])
use_assessments_unique[~pd.isna(use_assessments_unique['assessment_date'])].shape[0]/use_assessments_unique.shape[0]

### What parameters have caused the most impairments?

In [ ]:
use_assessments_unique[use_assessments_unique['parameterStatusName']=='Cause']['parameterName'].value_counts(normalize=True).iloc[0:10]

### What parameters have caused the most assessment units to be impaired, irrespective of number of use non-attainment declarations?

In [ ]:
culprits_unique = assessments.drop_duplicates(subset=['assessmentUnitIdentifier', 'reportingCycleText', 'assessment_date', 'parameterName'])

culprits_unique[culprits_unique['parameterStatusName']=='Cause']['parameterName'].value_counts(normalize=True).iloc[0:10]

### What uses have the highest rate of non-attainment?

In [ ]:
use_assessments_unique['useName'].value_counts(normalize=True).iloc[0:10]

Let's look at an example of one AUID x use x parameter combination over time:

In [ ]:
assessments[(assessments['assessmentUnitIdentifier']=='AL03140106-0302-202') &
            (assessments['useName']=='Contact Recreation') &
            (assessments['parameterName']=='AMMONIA, TOTAL')].sort_values(by='reportingCycleText').drop_duplicates()

Clearly, the fields `cycle_first_listed` and `cycleLastAssessedText` are misleading.

### Which states have the most and fewest reporting cycles present in ATTAINS?

In [ ]:
state_reporting_cycles = assessments.groupby('state_code').nunique()[['reportingCycleText']].sort_values('reportingCycleText', ascending=False)

state_reporting_cycles.head(15)

In [ ]:
state_reporting_cycles.tail(10)

# Actions
Source: `pull_actions.R`

In [ ]:
all_actions = pd.read_csv('all_actions.csv', parse_dates=['completionDate', 'TMDLDate'], date_parser=lambda t: pd.to_datetime(t, errors='coerce')).drop('Unnamed: 0', axis=1).drop_duplicates()
all_actions_permit_data = pd.read_csv('all_actions_permit_data.csv').drop('Unnamed: 0', axis=1).drop_duplicates()

### How many unique TMDL actions do we have?

In [ ]:
len(all_actions.drop_duplicates(subset='actionIdentifier'))

How does this compare to the number of unique actions in our assessments table?

In [ ]:
len(assessments.drop_duplicates(subset='associatedActionIdentifier'))

### How many unique AUs are represented in the actions data?

In [ ]:
len(all_actions.drop_duplicates(subset='assessmentUnitIdentifier'))

### Which states have the most TMDL actions?

In [ ]:
all_actions.drop_duplicates(subset='actionIdentifier')['state_code'].value_counts(normalize=True)

### Which states have the highest and lowest rates of TMDL actions per AUID?

In [ ]:
TMDL_rates = pd.DataFrame(all_actions.drop_duplicates(
    subset=['actionIdentifier'])['state_code'].value_counts())
TMDL_rates = TMDL_rates.reset_index().rename(columns={'index': 'state', 'state_code': '# TMDLs'})
TMDL_rates = TMDL_rates[TMDL_rates['state'] != 'GU']
TMDL_rates['AUs'] = TMDL_rates['state'].apply(lambda x: AU_counts[x])
TMDL_rates['TMDL rate'] = TMDL_rates['# TMDLs'] / TMDL_rates['AUs']
TMDL_rates.sort_values(by='TMDL rate', ascending=False).head(10)

In [ ]:
TMDL_rates.sort_values(by='TMDL rate', ascending=False).tail(10)

### What is the temporal distribution of TMDLs?

Based on a document here: https://www.exchangenetwork.net/schema/ATTAINS/1/ATTAINS_DET_v1.0a.xlsx , `completionDate` refers to the planned TMDL date, and `TMDLDate` refers to the actual date the TMDL was approved by EPA.

In [ ]:
plt.title('Planned TMDL Date')
plt.hist(all_actions.drop_duplicates(subset='actionIdentifier')['completionDate'], bins=50);

In [ ]:
plt.title('Actual TMDL Date')
plt.hist(all_actions.drop_duplicates(subset='actionIdentifier')['TMDLDate'], bins=50);

In [ ]:
import numpy as np
delays = (all_actions.drop_duplicates(subset='actionIdentifier')['TMDLDate'] -
          all_actions.drop_duplicates(subset='actionIdentifier')['completionDate']) / dt.timedelta(
    days=1)
plt.figure(figsize=(12, 8))
plt.xscale("log")
plt.hist(delays, bins=np.logspace(np.log10(1),np.log10(10000.0), 50))
plt.title('Days between planned TMDL Date and actual TMDL Date')
plt.ylabel('# TMDLs')
plt.xlabel('Days')

### Which pollutants most often lead to TMDLs?

In [ ]:
all_actions.drop_duplicates(subset=['actionIdentifier', 'pollutantName'])['pollutantName'].value_counts(normalize=True).iloc[0:10]

### Which combinations of state x pollutant are most frequent across all TMDLS?

In [ ]:
pd.DataFrame(all_actions.drop_duplicates(subset=['actionIdentifier', 'pollutantName'])[['pollutantName', 'state_code']].value_counts(normalize=True).iloc[0:10])

### For how many TMDL actions do we have one or more NPDES permit ID matches?

In [ ]:
all_actions_permit_data.dropna().drop_duplicates('actionIdentifier').shape[0]

### What is the distribution of these TMDLs across states?

In [ ]:
all_actions_permit_data.dropna().drop_duplicates('actionIdentifier')['NPDESIdentifier'].apply(lambda x: x[0:2]).value_counts(normalize=True)

### What is the time delay between a water body appearing on the 303(d) list and receiving a TMDL?
Let's try to answer this question for a few states first before looking at the whole country.

In [ ]:
# These states were selected based on having a high number of unique reporting cycles present in their assessments table
states = ['NM', 'VA', 'MS', 'CT']
for state in states:
    state_actions = all_actions[(all_actions['state_code'] == state) & (all_actions['TMDLDate']>=pd.to_datetime('2003-01-01'))].dropna(subset=['assessmentUnitIdentifier', 'pollutantName'])
    state_assessments = assessments[assessments['state_code'] == state].dropna(
        subset=['assessmentUnitIdentifier', 'parameterName'])

    # Data cleaning to account for changes in parameter categorization over time
    state_assessments['parameterName'] = state_assessments['parameterName'].apply(lambda x: x.replace('SULFATES', 'SULFATE'))
    state_assessments['parameterName'] = state_assessments['parameterName'].apply(
        lambda x: x.replace('TEMPERATURE, WATER', 'TEMPERATURE'))
    state_assessments['parameterName'] = state_assessments['parameterName'].apply(
        lambda x: x.replace('ESCHERICHIA COLI (E. COLI)', 'PATHOGENS'))
    state_assessments['parameterName'] = state_assessments['parameterName'].apply(
        lambda x: x.replace('FECAL COLIFORM', 'PATHOGENS'))
    state_assessments['parameterName'] = state_assessments['parameterName'].apply(
        lambda x: x.replace('ENTEROCOCCUS BACTERIA', 'ENTEROCOCCUS'))
    state_actions['pollutantName'] = state_actions['pollutantName'].apply(
        lambda x: x.replace('FECAL COLIFORM', 'PATHOGENS'))
    state_actions['pollutantName'] = state_actions['pollutantName'].apply(
        lambda x: x.replace('ESCHERICHIA COLI (E. COLI)', 'PATHOGENS'))

    # Determine first appearance on a 303(d) list in ATTAINS
    appearance_dates = state_assessments.groupby(['assessmentUnitIdentifier', 'parameterName'], as_index=False).min()[
        ['assessmentUnitIdentifier', 'parameterName', 'reportingCycleText']]
    appearance_dates.rename(columns={'reportingCycleText': 'appearance_date', 'parameterName': 'pollutantName'},
                               inplace=True)
    appearance_dates['appearance_date'] = pd.to_datetime(appearance_dates['appearance_date'])

    # Merge on AU and pollutant
    merged = state_actions.merge(appearance_dates, on=['assessmentUnitIdentifier', 'pollutantName'], how='inner')

    # Calculate delay time
    delays = (merged.drop_duplicates(subset=['assessmentUnitIdentifier', 'pollutantName'])['TMDLDate'] - merged.drop_duplicates(subset=['assessmentUnitIdentifier', 'pollutantName'])[
              'appearance_date']) / dt.timedelta(days=365)

    # Plot
    plt.figure(figsize=(10, 6))
    plt.hist(delays, bins=50);
    plt.axvline(x=0, linestyle='--', c='silver')
    plt.ylabel('# of AU x pollutant combinations')
    plt.xlabel('Years')
    plt.title(state + ': Delays between first impairment in ATTAINS and TMDL Date \n (n=' + str(len(delays)) + ' combos, ' + str(len(merged['assessmentUnitIdentifier'].unique())) + ' assessment units (AUs). TMDLs 2003 or later included.)')

### How many AUs scheduled to receive a TMDL actually received one?

In [ ]:
assessments.head()

In [ ]:
len(merged[(merged['cycle_scheduled_for_TMDL'] < merged['reportingCycleText']) & (~pd.isna(merged['TMDLDate']))].drop_duplicates(subset='assessmentUnitIdentifier'))

That's about half.